``` Chat With Database ```

In [ ]:
! pip install llama-index-tools-database
! pip install llama-index-llms-lmstudio

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
import os

ollama_host = os.getenv("OLLAMA_API")
ollama_model = os.getenv("OLLAMA_MODEL")
openai_api_key = os.getenv("OPENAI_API_KEY")
embedding = OllamaEmbedding(model_name=ollama_model, base_url=ollama_host)
llm = Ollama(model=ollama_model, base_url=ollama_host, request_timeout=120)
openai = OpenAI(api_base="http://127.0.0.1:3333/v1", api_key=openai_api_key)
Settings.llm = openai
Settings.embed_model = embedding
# print(openai.complete("who is lionel messi?"))

```chat agent```

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.tools.database import DatabaseToolSpec
db_tools = DatabaseToolSpec(uri="sqlite:///database//employees.db")
tools = db_tools.to_tool_list()

step_engine = OpenAIAgentWorker.from_tools(llm= openai, 
                                           tools=tools, 
                                           verbose=True)
agent = AgentRunner(step_engine)
input =  (
    "Find the highest paid HR employee and write "
    "them an email announcing a bonus"
)

task = agent.create_task(input)
step_output = agent.run_step(task.task_id)

while not step_output.is_last:
    step_output = agent.run_step(task.task_id)

response = agent.finalize_response(task.task_id)
print(response)


```chat memory```

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.memory import ChatMemoryBuffer

try:
    chat_store = SimpleChatStore.from_persist_path(
        persist_path="chat_memory.json"
    )
except FileNotFoundError:
    chat_store = SimpleChatStore()


memory = ChatMemoryBuffer.from_defaults(
    token_limit=2000,
    chat_store=chat_store,
    chat_store_key="user_X"
    )  

chat_engine = SimpleChatEngine.from_defaults(memory=memory)
while True:
    user_message = input("You: ")
    if user_message.lower() == 'exit':
        print("Exiting chat...")
        break
    response = chat_engine.chat(user_message)
    print(f"Chatbot: {response}")

chat_store.persist(persist_path="chat_memory.json")


```condense questions```

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.llms import ChatMessage

documents = SimpleDirectoryReader("files").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine=index.as_query_engine()
chat_history = [
    ChatMessage(
        role="user",
        content="Arch of Constantine is a famous"
        "building in Rome"
    ),
    ChatMessage(
        role="user",
        content="The Pantheon should not be "
        "regarded as a famous building"
    ),
]

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    chat_history=chat_history
)
response = chat_engine.chat(
    "What are two of the most famous structures in ancient Rome?"
)
print(response)


```context chat engine```

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
docs = SimpleDirectoryReader(input_dir="files").load_data()
index = VectorStoreIndex.from_documents(docs)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt=(
        "You’re a chatbot, able to talk about "
        "general topics, as well as answering specific "
        "questions about ancient Rome."
    ),
)
chat_engine.chat_repl()
